# OldBaseline_KLL_MLT22 Git revision: End2End_demo.ipynb (EXPLANATION)

## Environment Setup and Library Import

In [ ]:
import argparse
import pandas as pd
import numpy as np
import math
import h5py
from sklearn.model_selection import train_test_split
import joblib
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import sys
import gc

# import setGPU
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

from datetime import datetime
from tensorboard import program
import os
import pathlib
import matplotlib as mpl
import matplotlib.pyplot as plt
try:
    import mplhep as hep
    hep.style.use(hep.style.ROOT)
    print("Using MPL HEP for ROOT style formating")
except:
    print("Instal MPL HEP for style formating")
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["#DB4437", "#4285F4", "#F4B400", "#0F9D58", "purple", "goldenrod", "peru", "coral","turquoise",'gray','navy','m','darkgreen','fuchsia','steelblue']) 
from autoencoder_classes import AE,VAE

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from neptunecontrib.monitoring.keras import NeptuneMonitor
from losses import mse_split_loss, radius, kl_loss
from functions import make_mse_loss_numpy, save_model , make_mse_loss, mse_cart_loss, mse_cart_loss_numpy
from sklearn.metrics import roc_curve, auc


from data_preprocessing import prepare_data
from model import build_AE, build_VAE, Sampling


def return_total_loss(loss, bsm_t, bsm_pred):
    total_loss = loss(bsm_t, bsm_pred.astype(np.float32))
    return total_loss

Let's see the next libraries and dependencies that are being imported:
* `h5py`: For reading and writing **HDF5** files.
* `sklearn.model_selection`: For splitting the dataset into training and test sets.
* `joblib`: For saving and loading Python objects efficiently.
* `pickle`: For **serializing and de-serializing Python objects**.
* `sklearn.preprocessing`: For scaling and transforming data.
* `sys`: For system-specific parameters and functions.
* `gc`: For garbage collection.
* `tensorflow.keras as keras`: For high-level API to build and train models.
* `Model, Input, Dense, Lambda, ...`: Core components and layers from Keras for building neural networks.
* `K`: Keras backend for lower-level operations.
* `tf.keras.mixed_precision.set_global_policy('mixed_float16')`: For setting **mixed precision policy** to improve performance.
* `tensorboard`: For visualizing the training process.
* `os, pathlib`: For file and directory operations.
* `mplhep`: For high-energy physics plotting style (ROOT style).
* `EarlyStopping, ReduceLROnPlateau, TerminateOnNaN`: Keras **callbacks** for controlling the training process.
* `def return_total_loss(loss, bsm_t, bsm_pred)`: A utility function that computes the total loss given a loss function, true labels `bsm_t`, and predicted labels `bsm_pred`.
* ***Custom modules***
    * `autoencoder_classes`: Custom module containing the definitions for AE and VAE classes.
    * `NeptuneMonitor`: Custom callback for integrating with Neptune for experiment tracking.
    * `losses`: Custom module containing loss functions.
    * `functions`: Custom module containing utility functions.
    * `sklearn.metrics`: For computing ROC curves and AUC metrics.
    * `data_preprocessing`: Custom module for preparing the data.
    * `model`: Custom module for building AE, VAE models, and the Sampling layer.

<span style="color: red;"> Key task: Research about all new packages and dependencies that were called. </span>


In [ ]:
####configuration####
input_qcd="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-ZB-h5-extended-v2/ZB_preprocessed.h5"
input_bsm="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-h5-extended-v2-120X/BSM_preprocessed.h5"
events = 2000000
load_pickle=True
train=False
input_pickle="data.pickle"
output_pfile="data.pickle"
output_model_h5='model.h5'
output_model_json='model.json'
output_history='history.h5'
output_result='results.h5'
model_type='VAE'
latent_dim=3
batch_size= 1024
n_epochs = 150

This configuration block sets up various parameters and file paths that will be used later in the demo notebook.
* ***File Paths and Data***
    * `input_qcd`: Path to the processed QCD dataset stored in HDF5 format. This file likely contains QCD events that will be used either for training, validation, or testing the model.
    * `input_bsm`: Path to the preprocessed BSM dataset stored in HDF5 format. This file contains BSM events for similar purposes as the QCD file.
    * These files are likely located on a distributed system used in HEP experiments, specifically on the EOS large-scale storage system solution used at CERN. `/eos/uscms/` suggests they are part of the US CMS storage at CERN. At lxplus, you can access to the EOS environment through `xrdfs eospublic.cern.ch`.
    
     <span style="color: red;"> Key task: Research about eospublic, how can I access to jngadiub files? </span>
* ***Data Handling***
    * `events` specifies that 2 millions events will be loaded from the dataset. It is the number of events to be used.
    * `load_pickle`: A boolean flag to indicate whether to load preprocessed data from a pickle file. If `True`, the notebook will load data from a pickle file instead of preprocessing it from the raw HDF5 files.
    * `train`: A boolean flag indicating whether to train the model. If `False`, the notebook might skip the training step and load a pre-trained model instead.
* ***File Paths for Intermediate and Output Data***
    * `input_pickle`: Path to the pickle file containing preprocessed data. This file will be loaded if `load_pickle` is `True`.
    * `output_pfile`: Path to save the preprocessed data in pickle format. This file will be created or overwritten to save the preprocessed data.
    * `output_model_h5`: Path to save the trained model in HDF5 format. The trained model will be saved here if training is performed.
    * `output_model_json`: Path to save the model architecture in JSON format. The model architecture will be saved in a JSON file for potential reconstruction later.
    * `output_history`: Path to save the training history. The training history, which includes loss and metrics over epochs, will be saved in this HDF5 file.
    * `output_result`: Path to save the results of the model evaluation.The evaluation results, such as predictions or performance metrics, will be saved in this HDF5 file.
* ***Model Configuration***
    * `model_type`: The type of model to be used, either `AE` for Autoencoder or `VAE` for Variational Autoencoder.
    * `latent_dim`: The dimensionality of the latent space.
* ***Training Parameters***
    * `batch_size`: The number of samples per gradient update. The model will process that amount of samples at a time during training.
    * `n_epochs`: The number of epochs to train the model.

In [ ]:
if(load_pickle):
    if(input_pickle==''):
        print('Please provide input pickle files')
    with open(input_pickle, 'rb') as f:
        X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler, bsm_labels = pickle.load(f)
        bsm_types = [ 'GluGluHToTauTau_M125',
		      'GluGluToHHTo4B_cHHH1',
		      'GluGluToHHTo4B_cHHH5',
		      'HTo2LongLivedTo4b_MH1000_MFF450_CTau100m',
		      'HTo2LongLivedTo4b_MH1000_MFF450_CTau10m',
		      'HTo2LongLivedTo4b_MH125_MFF12_CTau9m',
		      'HTo2LongLivedTo4b_MH125_MFF12_CTau0p9m', 
		      'HTo2LongLivedTo4b_MH125_MFF25_CTau15m',
		      'HTo2LongLivedTo4b_MH125_MFF25_CTau1p5m', 
		      'HTo2LongLivedTo4b_MH125_MFF50_CTau30m',
		      'HTo2LongLivedTo4b_MH125_MFF50_CTau3m',
		      'HTo2LongLivedTo4b_MH250_MFF120_CTau10m',
		      'HTo2LongLivedTo4b_MH250_MFF120_CTau1m',
		      'HTo2LongLivedTo4b_MH250_MFF60_CTau1m',
		      'HTo2LongLivedTo4b_MH350_MFF160_CTau10m', 
		      'HTo2LongLivedTo4b_MH350_MFF160_CTau1m',
		      'HTo2LongLivedTo4b_MH350_MFF160_CTau0p5m',
		      'HTo2LongLivedTo4b_MH350_MFF80_CTau10m',
		      'HTo2LongLivedTo4b_MH350_MFF80_CTau1m',
		      'HTo2LongLivedTo4b_MH350_MFF80_CTau0p5m',
		      'HTo2LongLivedTo4mu_MH1000_MFF450_CTau10m',
		      'HTo2LongLivedTo4mu_MH125_MFF12_CTau0p9m',
		      'HTo2LongLivedTo4mu_MH125_MFF25_CTau1p5m',
		      'HTo2LongLivedTo4mu_MH125_MFF50_CTau3m',
		      'SUSYGluGluToBBHToBB_M1200',
		      'SUSYGluGluToBBHToBB_M120',
		      'SUSYGluGluToBBHToBB_M350',
		      'SUSYGluGluToBBHToBB_M600',
		      'TprimeBToTH_M650',
		      'VBFHHTo4B_CV_1_C2V_2_C3_1',
		      'VBFHToInvisible_M125',
		      'VBFHToTauTau_M125',
		      'VectorZPrimeGammaToQQGamma_M10_GPt75',
		      'VectorZPrimeToQQ_M100_Pt300',
		      'VectorZPrimeToQQ_M200_Pt300',
		      'haa4b_ma60',
		      'haa4b_ma15',
		      'haa4b_ma15_powheg',
		      'TT'
                    ]
else:
    if(input_qcd==''or input_bsm==''):
        print('Please provide input H5 files')
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler, bsm_labels = prepare_data(input_qcd, input_bsm, events, 'Cartesian', output_pfile,True)

This code block shows how the script either loads preprocessed data from a pickle file or reads and preprocesses raw data from H5 files, depending on the `load_pickle` flag. This allows for flexibility in the data loading process, making it easier to switch between using preprocessed data and raw data.

* In principle, the code above checks `load_pickle`:
    * If `load_pickle` is `True`:
        * Check if `input_pickle` if empty. If it is, the program prints a message asking to provide input pickle files.
        * If it is not empty, inside `with open(input_pickle, 'rb') as f` the pickle file in binary read mode (`rb`) will be opened. `pickle.load(f)` reads the data from the file and deserializes into the listed variables.
    * If `load_pickle` is `False`:
        * Check if either `input_qcd` or `input_bsm` is empty. If either of the inputs is an empty string, it prints a message asking to provide input H5 files.
        * Once you provide the input files, the customized function `prepare_data()` will read and preprocesses them, and returns several variables as `X_train_flatten, X_train_scaled, ...`.
* It is good to know that:
    * **Pickle File**: A pickle file is used to store Python objects in a serialized format. This is useful for saving preprocessed data and loading it later without having to preprocess it again.
    * **H5 file**: An H5 file is a hierarchical data format file used to store large amounts of data.
    * **Data Preprocessing**: The `prepare_data` function handles the conversion and preprocessing of raw data from H5 files into a format suitable for training and testing machine learning models. This includes tasks like flattening multidimensional arrays and scaling data to a specific range.

In [ ]:
if(train):
    if(model_type=='AE'):
        autoencoder = build_AE(X_train_flatten.shape[-1],latent_dim)
        model = AE(autoencoder)
        model.compile(optimizer=keras.optimizers.Adam(lr=0.001))

        callbacks=[]
        callbacks.append(ReduceLROnPlateau(monitor='val_loss',  factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
        callbacks.append(TerminateOnNaN())
        callbacks.append(NeptuneMonitor())
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=10, restore_best_weights=True))

    elif(model_type=='VAE'):
        encoder, decoder = build_VAE(X_train_flatten.shape[-1],latent_dim)
        model = VAE(encoder, decoder, mse_cart_loss)
        model.compile(optimizer=keras.optimizers.Adam())

        callbacks=[]
        callbacks.append(ReduceLROnPlateau(monitor='val_loss',  factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
        callbacks.append(TerminateOnNaN())
        callbacks.append(NeptuneMonitor())
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=10, restore_best_weights=True))

    print("Training the model")

    history = model.fit(X_train_flatten, X_train_scaled,
                        epochs=n_epochs,
                        batch_size=batch_size,
                        validation_split=0.2,
                        callbacks=callbacks)

    del X_train_flatten, X_train_scaled
    gc.collect()

* The initial condition checks if the `train` flag is set to `True`. If so, the model training process will be initiated. 
    * ***Building the Autoencoder***:
        * `build_AE(X_train_flatten.shape[-1], latent_dim)` creates an AE model. The input size is determined by the shape of `X_train_flatten` and the latent dimension is specified by `latent_dim`.
        * `model = AE(autoencoder)` wraps the autoencoder in a class (presumably defined elsewhere, `AE()` is a function defined in a customized library that was called in the import statement code block using `autoencoder_classes`) that includes additional methods or properties specific to the project.
        * `model.compile(optimizer=keras.optimizers.Adam(lr=0.001))` compiles the model using the Adam optimizer with a learning rate of 0.001.
        * ***Setting up Callbacks***
            * `ReduceLROnPlateau`: Reduces the learning rate by a factor of 0.1 if the validation loss doesn't improve for 2 epochs, with a minimum delta of 0.0001, a cooldown period of 2 epochs, and a minimum learning rate of 1e-6.
            * `TerminateOnNaN()`: Stops training if any NaN values are encountered.
            * `NeptuneMonitor()`: Tracks the training process using `Neptune` for monitoring and logging.
            * `EarlyStopping`: Stops training if the validation loss doesn't improve for 10 epochs and restores the best weights observed during training.
    * ***Building the Variational Autoencoder***
        * `build_VAE(X_train_flatten.shape[-1], latent_dim)`: Creates the encoder and decoder parts of a VAE model.
        * `model = VAE(encoder, decoder, mse_cart_loss)`: Initializes a VAE model with the encoder, decoder, and a custom loss function `mse_cart_loss`.
        * `model.compile(optimizer=keras.optimizers.Adam())`: Compiles the model using the Adam optimizer with default settings.
        * Callbacks were setting up in the same way as in the AE case.
    * ***Training The model***
        * `model.fit(...)`: Trains the model using the training data (`X_train_flatten and X_train_scaled`) for a specified number of epochs (`n_epochs`) and batch size (`batch_size`).
            * `validation_split=0.2`: Reserves 20% of the training data for validation.
            * `callbacks=callbacks`: Uses the callbacks defined earlier to optimize and manage the training process effectively.
    * ***Cleanup***
        * `del X_train_flatten, X_train_scaled`: Deletes the training data from memory to free up space.
        * `gc.collect()`: Invokes the garbage collector to free up unreferenced memory.



In [ ]:
if(train):
    if(output_model_h5!=''):
        if(model_type=='VAE'):
            model.save(os.path.join(os.getcwd(),output_model_h5.split('.')[0]))
        else:
            model_json = autoencoder.to_json()
            with open(output_model_json, 'w') as json_file:
                json_file.write(model_json)
            autoencoder.save_weights(output_model_h5)
            print("Saved model to disk")


    if(output_history!=''):
        with open(output_history, 'wb') as f:
            pickle.dump(history.history, f)
        print("Saved history to disk")



    # Plot training & validation loss values
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    print(hist.tail())
    plt.plot(hist.index.to_numpy(),hist['loss'],label='Loss')
    plt.plot(hist.index.to_numpy(),hist['val_loss'],label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.legend()
    plt.savefig('history.pdf')
    # plt.show()

* `if(train)` checks if the `train` flag is set to `True`. If so, the conde inside this block will execute.
    * `if(output_model_h5 != '')` chekcs if `output_model_h5` is specified, i.e, this condition checks if the `output_model_h5` string is not empty, indicating that a path to save the model is specified.
        * ***Save the VAE Model***: If the model type is VAE, the model is saved using the TensorFlow/Keras save method. `os.path.join(os.getcwd(), output_model_h5.split('.')[0])` creates the path to save the model in the current working directory. The `split('.')[0]` part removes the file extension from `output_model_h5`.
        * ***Save the AE Model***: If the model type is AE, the autoencoder's architecture is saved as a JSON file. `autoencoder.to_json()` converts the model architecture to a JSON string.
            * `with open(output_model_json, 'w') as json_file` opens the file specified by `output_model_json` in write mode. `json_file.write(model_json)` then writes the JSON string to the file.
            * `autoencoder.save_weights(output_model_h5)` saves the weights of the autoencoder model to the file specified by `output_model_h5`.
    * ***Saving the Training History***
        * `if(output_history != '')` checks if `output_history` is specified, i.e, it checks if the `output_history` string is not empty, indicating that a path to save the training history is specified.
        * `with open(output_history, 'wb') as f` opens the file specified by `output_history` in write-binary mode.
        * `pickle.dump(history.history, f)` serializes the training history dictionary and writes it to the file.
    * ***Plotting the Training and Validation Loss***
        * `hist = pd.DataFrame(history.history)` Converts the training history dictionary to a pandas DataFrame.
        * `hist['epoch'] = history.epoch`: Adds an `epoch` column to the DataFrame, containing the epoch numbers.
        * `print(hist.tail())`: Prints the last few rows of the DataFrame to the console.
        * `plt.plot(hist.index.to_numpy(), hist['loss'], label='Loss')`: Plots the training loss over epochs.
        * `plt.plot(hist.index.to_numpy(), hist['val_loss'], label='Val Loss')`: Plots the validation loss over epochs.
        * `plt.yscale('log')`: Sets the y-axis scale to logarithmic.
        * `plt.legend()`: Adds a legend to the plot to differentiate between the training and validation loss.
        * `plt.savefig('history.pdf')`: Saves the plot as a PDF file named `history.pdf`.


In [ ]:
#load model
model_dir = output_model_h5.split('.')[0]
if(model_type=='AE'):
    with open(model_dir+"/model.json", 'r') as jsonfile: config = jsonfile.read()
    ae = tf.keras.models.model_from_json(config)    
    ae.load_weights(model_dir+"/model.h5")
    ae.summary()
    model = AE(ae)
elif(model_type=='VAE'):
    encoder, decoder = VAE.load(model_dir, custom_objects={'Sampling': Sampling})
    encoder.summary()
    decoder.summary()
    model = VAE(encoder, decoder, mse_cart_loss)

The code block above is responsible for loading a previously saved model.
* ***Define the Model Directory***
    * `model_dir = output_model_h5.split('.')[0]`: This line takes the `output_model_h5` string, splits it at the period `.` character, and takes the first part. This effectively removes the file extension, leaving just the directory or base filename. For example, if `output_model_h5` is `model.h5`, `model_dir` will be `model`.
* ***Check Model Type***
    * `if(model_type=='AE')` checks if the `model_type` is set to `AE`.
        * `with open(model_dir+"/model.json", 'r') as jsonfile` opens the JSON file containing the model architecture in read mode.
        * `config = jsonfile.read()` reads the entire file content into the `config` variable.
        * `ae = tf.keras.models.model_from_json(config)` uses the JSON configuration to recreate the model architecture.
        * `ae.load_weights(model_dir+"/model.h5")` loads the weights into the model from the specified H5 file.
        * `ae.summary()` prints a summary of the model architecture to the console.
        * `model = AE(ae)` wraps the loaded model into an `AE` object.
    * `elif(model_type=='VAE')` checks if the model_type is set to `VAE` (Variational Autoencoder).
        * `encoder, decoder = VAE.load(model_dir, custom_objects={'Sampling': Sampling})` uses the custom `Sampling` layer to load the encoder and decoder models from the specified directory.
        * `model = VAE(encoder, decoder, mse_cart_loss)` creates a `VAE` object using the loaded encoder and decoder models, and specifies the loss function `mse_cart_loss`.

In [ ]:
from end2end import get_results
data_file = input_pickle
outdir = output_model_h5.split('.')[0]
if not load_pickle: data_file = output_pfile
results = get_results(input_qcd,input_bsm,data_file,outdir,events,model_type,latent_dim,'Cartesian')
print(results.keys())

* `get_results`, imported from `end2end` is responsible for loading a trained model and evaluating it on test data to produce results and return them in a dictionary format.
* `data_file` is set to the value of `input_pickle` (path to the pickle file that contains preprocessed data), which will be used to load the data for evaluation. 
* `outdir` is set to the name of the directory where the model output will be stored(basically it is `model_dir`).
* If `load_pickle` is `False` (i.e, there is not a input pickle file with preprocessed data), `data_file` will be set to `output_pfile` (file created to save the preprocessed data in pickle format) instead of `input_pickle`.
* `get_results(...,'Cartesian')` refers to the normalization strategy to be used. 
* `print(results.keys())` gives an overview of the different datasets or metrics that were evaluated and stored in the results dictionary.

In [ ]:
#select only a few signals to check performance
results_small = {}
keys = ['QCD','TT','haa4b_ma15_powheg','GluGluToHHTo4B_cHHH1','GluGluHToTauTau_M125']
for k in keys:
    results_small[k] = results[k]
results = results_small
del results_small

The code block above selects a subset of signales from the full set of results and stores them in a new dictionary.
* `results_small` is initialized as an empty dictionary. This will store only a subset of the original results.
* `keys` is a list containing the names of the signals that you want to keep in `results_small`. The code within to `for k in keys` filters the `results dictionary` to only include the entries specified in `keys`. Then, `results` is reasigned to reference `results_small`
* `del results_small` deletes the temporary `results_small` to free up memory.


In [ ]:
min_loss,max_loss=1e5,0
if(model_type=='VAE'):
    min_tloss,max_tloss=1e5,0
    min_r,max_r=1e5,0
for key in results.keys():
    # if(key=='QCD'): continue
    results[key]['loss'][results[key]['loss'][:] == np.inf] = 0
    results[key]['total_loss'][results[key]['total_loss'][:] == np.inf] = 0
    results[key]['radius'][results[key]['radius'][:] == np.inf] = 0
    results[key]['kl_loss'][results[key]['kl_loss'][:] == np.inf] = 0
    if(np.min(results[key]['loss'])<min_loss): min_loss = np.min(results[key]['loss'])
    if(np.max(results[key]['loss'])>max_loss): max_loss = np.max(results[key]['loss'])
    if(model_type=='VAE'):
        if(np.min(results[key]['total_loss'])<min_tloss): min_tloss = np.min(results[key]['total_loss'])
        if(np.max(results[key]['total_loss'])>max_tloss): max_tloss = np.max(results[key]['total_loss'])
        # if(max_tloss>np.mean(results[key]['total_loss'])+10*np.std(results[key]['total_loss'])): max_tloss = np.mean(results[key]['total_loss'])+10*np.std(results[key]['total_loss'])
        print(key,"Total_loss",np.mean(results[key]['total_loss'])+10*np.std(results[key]['total_loss']))
        if(np.min(results[key]['radius'])<min_r): min_r = np.min(results[key]['radius'])
        if(np.max(results[key]['radius'])>max_r): max_r = np.max(results[key]['radius'])
        # if(max_r>np.mean(results[key]['radius'])+10*np.std(results[key]['radius'])): max_r = np.mean(results[key]['radius'])+10*np.std(results[key]['radius'])
        print(key,"radius",np.mean(results[key]['radius'])+10*np.std(results[key]['radius']))
print(min_loss,max_loss)
print(min_tloss,max_tloss)
print(min_r,max_r)

In [ ]:
tag='vtest'
print("Plotting the results")
bins_=np.linspace(min_loss,max_loss,100)
plt.figure(figsize=(10,10))
for key in results.keys():
    if('TT' in key): key_ = 'TTbar'
    if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
    if('HH' in key): key_ = 'SM HH -> 4b'
    if('TauTau' in key): key_ = 'SM H -> 2tau'    
    if(key=='QCD'): plt.hist(results[key]['loss'],label=key,histtype='step',bins=bins_,color='black',linewidth=2,density=True)
    else: plt.hist(results[key]['loss'],label=key_,histtype='step',bins=bins_,density=True)
plt.legend(fontsize='x-small')
plt.yscale('log')
plt.xlabel('MSE Loss')
plt.ylabel('Density')
plt.title('MSE Loss distribution')
plt.savefig('{outdir}/MSE_loss_{model_type}.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))

# plt.show()

if(model_type=='VAE'):

    bins_=np.linspace(min_tloss,32673,100)
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'        
        if(key=='QCD'): plt.hist(results[key]['total_loss'],label=key,histtype='step',bins=bins_,color='black',linewidth=2,density=True)
        else: plt.hist(results[key]['total_loss'],label=key_,histtype='step',bins=bins_,density=True)
    plt.legend(fontsize='x-small')
    plt.yscale('log')
    plt.xlabel('Total Loss')
    plt.ylabel('Density')
    plt.title('Total Loss distribution')
    plt.savefig('{outdir}/Total_loss_{model_type}.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))

    bins_=np.linspace(min_r,10,100)
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'           
        if(key=='QCD'): plt.hist(results[key]['radius'],label=key,histtype='step',bins=bins_,color='black',linewidth=2,density=True)
        else: plt.hist(results[key]['radius'],label=key_,histtype='step',bins=bins_,density=True)
    plt.legend(fontsize='x-small')
    plt.yscale('log')
    plt.xlabel('Radius')
    plt.ylabel('Density')
    plt.title('Radius distribution')
    plt.savefig('{outdir}/Radius_{model_type}.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))

    bins_=np.linspace(min_r,10,100)
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'           
        if(key=='QCD'): plt.hist(results[key]['kl_loss'],label=key,histtype='step',bins=bins_,color='black',linewidth=2,density=True)
        else: plt.hist(results[key]['kl_loss'],label=key_,histtype='step',bins=bins_,density=True)
    plt.legend(fontsize='x-small')
    plt.yscale('log')
    plt.xlabel('KL Loss')
    plt.ylabel('Density')
    plt.title('KL Loss distribution')
    plt.savefig('{outdir}/KL_loss_{model_type}.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))
    
#     for key in results.keys():
#         plt.figure(figsize=(10,10))
#         for i in range(latent_dim):
#             plt.hist(results[key]['mean_prediction'][:,i],bins=100,label='mean '+str(i),histtype='step', density=True,range=[-5,5])
#         plt.legend(fontsize='x-small')
#         plt.xlabel('Loss')
#         plt.ylabel('z')
#         plt.title(key+' mean Z distribution')
#         plt.savefig('mean_z_'+model_type+'_'+key+'_'+tag+'.pdf')
#         # plt.show()

#     for key in results.keys():
#         plt.figure(figsize=(10,10))
#         for i in range(latent_dim):
#             plt.hist(results[key]['logvar_prediction'][:,i],bins=100,label='logvar '+str(i),histtype='step', density=True,range=[-20,20])
#         plt.legend(fontsize='x-small')
#         plt.xlabel('Loss')
#         plt.ylabel('z')
#         plt.title(key+' logvar Z distribution')
#         plt.savefig('logvar_z_'+model_type+'_'+key+'_'+tag+'.pdf')
#         # plt.show()

In [ ]:
#make reconstruction plots
plt.figure(figsize=(10,10))
bkg_key='QCD'
bins_ = np.linspace(-50,50,100)
results[bkg_key]['target'] = results[bkg_key]['target'].reshape((results[bkg_key]['target'].shape[0],19,3))
test = results[bkg_key]['target'][:,1:9,:]
test = test.reshape(results[bkg_key]['target'].shape[0]*8,3)
mask0 = test[:,0]!=0
mask1 = test[:,1]!=0
mask2 = test[:,2]!=0
mask = mask0 + mask1 + mask2
test = test[mask]
results[bkg_key]['prediction'] = results[bkg_key]['prediction'].reshape((results[bkg_key]['prediction'].shape[0],19,3))
predict = results[bkg_key]['prediction'][:,1:9,:]
predict = predict.reshape(results[bkg_key]['prediction'].shape[0]*8,3)
predict = predict[mask]
plt.hist(test.flatten(), bins=bins_, alpha=0.5, label='X_test')
plt.hist(predict.flatten(), bins=bins_, alpha=0.5, label='X_predict')
plt.xlabel('Pi (GeV)')
plt.legend(loc='upper right')
plt.yscale('log')
plt.title('Muon, electon, and photon pt')
plt.savefig( '{outdir}/reco_{model_type}_leptons.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))

plt.figure(figsize=(10,10))
bkg_key='QCD'
bins_ = np.linspace(-50,50,100)
test = results[bkg_key]['target'][:,9:20,:]
test = test.reshape(results[bkg_key]['target'].shape[0]*10,3)
mask0 = test[:,0]!=0
mask1 = test[:,1]!=0
mask2 = test[:,2]!=0
mask = mask0 + mask1 + mask2
test = test[mask]
predict = results[bkg_key]['prediction'][:,9:20,:]
predict = predict.reshape(results[bkg_key]['prediction'].shape[0]*10,3)
predict = predict[mask]
plt.hist(test[:,0].flatten(), bins=bins_, alpha=0.5, label='X_test')
plt.hist(predict[:,0].flatten(), bins=bins_, alpha=0.5, label='X_predict')
plt.xlabel('Pi (GeV)')
plt.legend(loc='upper right')
plt.yscale('log')
plt.title('Jet pt and MET') 
plt.savefig('{outdir}/reco_{model_type}_jetmet.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))

In [ ]:
#make roc curves
signal_eff={}
scale_factor_nugun = 11245.6*2544. #2e34*6.92e-26 # Hz/cm^2 * cm^2 https://twiki.cern.ch/twiki/bin/view/CMS/PileupJSONFileforData#Recommended_cross_section
scale_factor_nugun = scale_factor_nugun/(10**6) # MHz

print("MSE Loss:")
plt.figure(figsize=(10,10))
for key in results.keys():
    if key=='QCD': continue
    if('TT' in key): key_ = 'TTbar'
    if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
    if('HH' in key): key_ = 'SM HH -> 4b'
    if('TauTau' in key): key_ = 'SM H -> 2tau'
    signal_eff[key]={}
    true_label = np.concatenate(( np.ones(results[key]['target'].shape[0]), np.zeros(results['QCD']['prediction'].shape[0]) ))
    pred_loss = np.concatenate(( results[key]['loss'], results['QCD']['loss'] ))
    fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
    roc_auc = auc(fpr_loss, tpr_loss)
    rate = fpr_loss*scale_factor_nugun
    signal_eff[key]['MSE_loss_5kHz']=tpr_loss[rate<0.005][-1]
    signal_eff[key]['MSE_loss_10Hz']=tpr_loss[rate<1e-5][-1]
    print(key_,"%.2f"%(tpr_loss[rate<0.005][-1]*100)+"%", '@ 5kHz')
    print(key_,"%.3f"%(tpr_loss[rate<1e-5][-1]*100)+"%", '@ 10 Hz')
    plt.plot(rate, tpr_loss, label=key_+' (AUC = %0.2f)' % roc_auc)
plt.plot([0, scale_factor_nugun], [0, 1], 'k--')
plt.xlim([1e-6, scale_factor_nugun])
plt.ylim([1e-6, 1.05])
plt.xlabel('Trigger Rate (MHz)')
plt.ylabel('Signal Efficiency')
plt.title('MSE Loss')
plt.axvline(0.005, color='red', linestyle='dashed', linewidth=1)
plt.axvline(10.0/10.0**6, color='green', linestyle='dashed', linewidth=1)
plt.legend(loc="lower right")
plt.xscale('log')
plt.yscale('log')
plt.savefig( '{outdir}/roc_curve_{model_type}_MSE_loss.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))
#plt.show()


if(model_type=='VAE'):
    
    print("\nTotal Loss:")
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if key=='QCD': continue
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'
        true_label = np.concatenate(( np.ones(results[key]['target'].shape[0]), np.zeros(results['QCD']['prediction'].shape[0]) ))
        pred_loss = np.concatenate(( results[key]['total_loss'], results['QCD']['total_loss'] ))
        fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
        roc_auc = auc(fpr_loss, tpr_loss)
        rate = fpr_loss*scale_factor_nugun
        signal_eff[key]['Total_loss_5kHz']=tpr_loss[rate<0.005][-1]
        signal_eff[key]['Total_loss_10Hz']=tpr_loss[rate<1e-5][-1]
        print(key_,"%.2f"%(tpr_loss[rate<0.005][-1]*100)+"%", '@ 5kHz')
        print(key_,"%.6f"%(tpr_loss[rate<1e-5][-1]*100)+"%", '@ 10 Hz')
        plt.plot(rate, tpr_loss, label=key_+' (AUC = %0.2f)' % roc_auc)
    plt.plot([0, scale_factor_nugun], [0, 1], 'k--')
    plt.xlim([1e-6, scale_factor_nugun])
    plt.ylim([1e-6, 1.05])
    plt.xlabel('Trigger Rate (MHz)')
    plt.ylabel('Signal Efficiency')
    plt.title('Total Loss')
    plt.axvline(0.005, color='red', linestyle='dashed', linewidth=1)
    plt.axvline(10.0/10.0**6, color='green', linestyle='dashed', linewidth=1)
    plt.legend(loc="lower right")
    plt.xscale('log')
    plt.yscale('log')
    plt.savefig( '{outdir}/roc_curve_{model_type}_Total_loss.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))
    # plt.show()

    print("\nRadius:")
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if key=='QCD': continue
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'
        true_label = np.concatenate(( np.ones(results[key]['target'].shape[0]), np.zeros(results['QCD']['prediction'].shape[0]) ))
        pred_loss = np.concatenate(( results[key]['radius'], results['QCD']['radius'] ))
        fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
        roc_auc = auc(fpr_loss, tpr_loss)
        rate = fpr_loss*scale_factor_nugun
        signal_eff[key]['Radius_5kHz']=tpr_loss[rate<0.005][-1]
        signal_eff[key]['Radius_10Hz']=tpr_loss[rate<1e-5][-1]
        print(key_,"%.2f"%(tpr_loss[rate<0.005][-1]*100)+"%", '@ 5kHz')
        print(key_,"%.6f"%(tpr_loss[rate<1e-5][-1]*100)+"%", '@ 10 Hz')
        plt.plot(rate, tpr_loss, label=key_+' (AUC = %0.2f)' % roc_auc)
    plt.plot([0, scale_factor_nugun], [0, 1], 'k--')
    plt.xlim([1e-6, scale_factor_nugun])
    plt.ylim([1e-6, 1.05])
    plt.xlabel('Trigger Rate (MHz)')
    plt.ylabel('Signal Efficiency')
    plt.title('Radius')
    plt.axvline(0.005, color='red', linestyle='dashed', linewidth=1)
    plt.axvline(10.0/10.0**6, color='green', linestyle='dashed', linewidth=1)
    plt.legend(loc="lower right")
    plt.xscale('log')
    plt.yscale('log')
    plt.savefig( '{outdir}/roc_curve_{model_type}_radius.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))
    # plt.show()

    print("\nKL Loss:")
    plt.figure(figsize=(10,10))
    for key in results.keys():
        if key=='QCD': continue
        if('TT' in key): key_ = 'TTbar'
        if('haa' in key): key_ = 'H -> aa -> 4b, ma = 15 GeV'
        if('HH' in key): key_ = 'SM HH -> 4b'
        if('TauTau' in key): key_ = 'SM H -> 2tau'
        true_label = np.concatenate(( np.ones(results[key]['target'].shape[0]), np.zeros(results['QCD']['prediction'].shape[0]) ))
        pred_loss = np.concatenate(( results[key]['kl_loss'], results['QCD']['kl_loss'] ))
        fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
        roc_auc = auc(fpr_loss, tpr_loss)
        rate = fpr_loss*scale_factor_nugun
        signal_eff[key]['KL_loss_5kHz']=tpr_loss[rate<0.005][-1]
        signal_eff[key]['KL_loss_10Hz']=tpr_loss[rate<1e-5][-1]
        print(key_,"%.2f"%(tpr_loss[rate<0.005][-1]*100)+"%", '@ 5kHz')
        print(key_,"%.6f"%(tpr_loss[rate<1e-5][-1]*100)+"%", '@ 10 Hz')
        plt.plot(rate, tpr_loss, label=key_+' (AUC = %0.2f)' % roc_auc)
    plt.plot([0, scale_factor_nugun], [0, 1], 'k--')
    plt.xlim([1e-6, scale_factor_nugun])
    plt.ylim([1e-6, 1.05])
    plt.xlabel('Trigger Rate (MHz)')
    plt.ylabel('Signal Efficiency')
    plt.title('KL Loss')
    plt.axvline(0.005, color='red', linestyle='dashed', linewidth=1)
    plt.axvline(10.0/10.0**6, color='green', linestyle='dashed', linewidth=1)
    plt.legend(loc="lower right")
    plt.xscale('log')
    plt.yscale('log')
    plt.savefig( '{outdir}/roc_curve_{model_type}_kl_loss.pdf'.format(outdir=os.path.join(os.getcwd(),output_model_h5.split('.')[0]),model_type=model_type))
    # plt.show()
    
signal_eff_pd = pd.DataFrame.from_dict(signal_eff).transpose()

In [ ]:
print(f_pd)